In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import kr_helper_funcs as kr
from sklearn.preprocessing import MinMaxScaler
import kr_helper_funcs as kr
from sklearn.metrics import classification_report, confusion_matrix

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
name = 'with_postcode'
fileName = "{}/lending-club-values.csv".format(name)
model = tf.keras.models.load_model('{}/lending-club.h5'.format(name))
X_train_unscaled = np.load("{}/X_train_unscaled.npy".format(name))
X_test_unscaled = np.load("{}/X_test_unscaled.npy".format(name))
y_train = np.load("{}/y_test.npy".format(name))
y_test = np.load("{}/y_test.npy".format(name))
df = pd.read_csv('{}/lending-club-df.csv'.format(name))
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train_unscaled)
X_test = scaler.transform(X_test_unscaled)

prev_scaled_row = None
cached_map_values = None
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train_unscaled)
X_test = scaler.transform(X_test_unscaled)

res = pd.read_csv(fileName)

In [3]:
df.drop('loan_repaid', axis=1).columns

Index(['loan_amnt', 'term', 'int_rate', 'installment', 'annual_inc', 'dti',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'mort_acc', 'pub_rec_bankruptcies', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2',
       'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5', 'D1', 'D2', 'D3', 'D4',
       'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5', 'G1',
       'G2', 'G3', 'G4', 'G5', 'verification_status_Source Verified',
       'verification_status_Verified', 'application_type_INDIVIDUAL',
       'application_type_JOINT', 'initial_list_status_w',
       'purpose_credit_card', 'purpose_debt_consolidation',
       'purpose_educational', 'purpose_home_improvement', 'purpose_house',
       'purpose_major_purchase', 'purpose_medical', 'purpose_moving',
       'purpose_other', 'purpose_renewable_energy', 'purpose_small_business',
       'purpose_vacation', 'purpose_wedding', 'OTHER', 'OWN', 'RENT', '05113',
       '11650', '22690', '29597', '30723', '48052', '7046

In [13]:
import eli5
from eli5.sklearn import PermutationImportance

# X_train_unscaled, X_test_unscaled, y_train, y_test
# train_x, val_x, train_y, val_y

perm = PermutationImportance(model).fit(X_test, y_test)
eli5.show_weights(perm, feature_names = df.drop('loan_repaid', axis=1).columns)

TypeError: If no scoring is specified, the estimator passed should have a 'score' method. The estimator <tensorflow.python.keras.engine.sequential.Sequential object at 0x00000252E0BDE588> does not.

In [ ]:
eli5.explain_weights(model)

In [9]:
from eli5.lime import TextExplainer
help(eli5)

Help on package eli5:

NAME
    eli5 - # -*- coding: utf-8 -*-

PACKAGE CONTENTS
    _decision_path
    _feature_importances
    _feature_names
    _feature_weights
    _graphviz
    base
    base_utils
    catboost
    explain
    formatters (package)
    ipython
    keras (package)
    lightgbm
    lightning
    lime (package)
    permutation_importance
    sklearn (package)
    sklearn_crfsuite (package)
    transform
    utils
    xgboost

DATA
    absolute_import = _Feature((2, 5, 0, 'alpha', 1), (3, 0, 0, 'alpha', 0...

VERSION
    0.10.1

FILE
    c:\users\dan39\anaconda3\envs\tensorflow\lib\site-packages\eli5\__init__.py




In [ ]:
eli5.show_prediction(model, X_test[0])

In [ ]:
history = pd.read_csv('{}/lending-club-history.csv'.format(name))
history_dict = history.to_dict('list')
kr.show_plots(history_dict)

In [ ]:
predictions = model.predict_classes(X_test)
print(classification_report(y_test, predictions))


In [ ]:
kr.plot_cm(y_test, predictions, ["unpaid", "paid"])

In [ ]:
res.head()


In [ ]:
sns.countplot(x="explainer",data=res[(res['nsamples'] == 'auto') ], hue = 'class_change')
plt.title("Number of rows predicted as the same class vs predicted as opposite class after columns have been neutralized based on the explainer's identified features")

In [ ]:
res.count()

In [ ]:
sns.countplot(x="no_features",data=res[(res['nsamples'] == 'auto') ], hue = 'class_change')

In [ ]:
avg_random = res[(res['nsamples'] == 'auto') & (res['explainer'] == 'random')].groupby("no_features")["confidence_diff"].mean() 
avg_shap = res[(res['nsamples'] == 'auto') & (res['explainer'] == 'shap')].groupby("no_features")["confidence_diff"].mean()
avg_lime = res[(res['nsamples'] == 'auto') & (res['explainer'] == 'lime')].groupby("no_features")["confidence_diff"].mean()

plt.figure(figsize=(10,4))
# plt.set_title('Training & Validation Loss')
no_of_features = list(range(0, len(avg_random) ))
plt.plot( no_of_features, avg_random, color='navy', marker='*', linestyle='-', label='Random')
plt.plot( no_of_features, avg_shap, color='green', marker='x', linestyle='-', label='Shap')
plt.plot( no_of_features, avg_lime, color='red', marker='o', linestyle='-', label='Lime')
plt.title("Average of confidence level change vs no of features neutralized. If no feature is altered the (old prediction - new prediction) == 0 should stay the same. The more features we change, the higher should the difference increase")
plt.legend(loc='best')

In [ ]:
# Same as the graph above but the plot is class specific, in this case for class 1. For class 0, there are not enough datapoints to plot the graph
original_class = 1
avg_random = res[(res['nsamples'] == 'auto') & (res['explainer'] == 'random') & (res['original_class'] == original_class)].groupby("no_features")["confidence_diff"].mean() 
avg_shap = res[(res['nsamples'] == 'auto') & (res['explainer'] == 'shap') & (res['original_class'] == original_class)].groupby("no_features")["confidence_diff"].mean()
avg_lime = res[(res['nsamples'] == 'auto') & (res['explainer'] == 'lime') & (res['original_class'] == original_class)].groupby("no_features")["confidence_diff"].mean()

plt.figure(figsize=(11,4))
# plt.set_title('Training & Validation Loss')
no_of_features = list(range(0, len(avg_random) ))
plt.plot( no_of_features, avg_random, color='navy', marker='*', linestyle='-', label='Random')
plt.plot( no_of_features, avg_shap, color='green', marker='x', linestyle='-', label='Shap')
plt.plot( no_of_features, avg_lime, color='red', marker='o', linestyle='-', label='Lime')
plt.legend(loc='best')

In [ ]:
res[res["original_class"] == 0].head(5)

In [ ]:

# avg = res[res['confidence_diff'] == 0.0].groupby("no_features")["confidence_diff"].count() / res.groupby("no_features")["confidence_diff"].count() # / res[(res['nsamples'] == 'auto') & (res['explainer'] == 'random')].groupby("no_features")["confidence_diff"].count()
avg_random = res[(res['confidence_diff'] < 0.01) & (res['nsamples'] == 'auto') & (res['explainer'] == 'random')].groupby("no_features")["confidence_diff"].count() / res[(res['nsamples'] == 'auto') & (res['explainer'] == 'random')].groupby("no_features")["confidence_diff"].count()
avg_shap = res[(res['confidence_diff'] < 0.01) & (res['nsamples'] == 'auto') & (res['explainer'] == 'shap')].groupby("no_features")["confidence_diff"].count() / res[(res['nsamples'] == 'auto') & (res['explainer'] == 'shap')].groupby("no_features")["confidence_diff"].count()
avg_lime = res[(res['confidence_diff'] < 0.01) & (res['nsamples'] == 'auto') & (res['explainer'] == 'lime')].groupby("no_features")["confidence_diff"].count() / res[(res['nsamples'] == 'auto') & (res['explainer'] == 'lime')].groupby("no_features")["confidence_diff"].count()

plt.figure(figsize=(10,4))
# plt.set_title('Training & Validation Loss')
no_of_features = list(range(0, len(avg_shap) ))
plt.plot( no_of_features, avg_random, color='navy', marker='*', linestyle='-', label='Random Confidence level change vs no of features')
plt.plot( no_of_features, avg_shap, color='green', marker='x', linestyle='-', label='Shap Confidence level change vs no of features')
plt.plot( no_of_features, avg_lime, color='red', marker='o', linestyle='-', label='Lime Confidence level change vs no of features')
plt.title("Percentage of rows where the new prediction favours the original class instead of decreasing its probability or has no impact. (old prediction - new prediction) <=0.1")
plt.legend(loc='best')

In [ ]:

avg_random = res[(res["no_features"]==0) & (res['explainer'] == 'random')].groupby("nsamples")["time"].mean() 
avg_shap = res[(res["no_features"]==0) & (res['explainer'] == 'shap')].groupby("nsamples")["time"].mean()
avg_lime = res[(res["no_features"]==0) & (res['explainer'] == 'lime')].groupby("nsamples")["time"].mean()

plt.figure(figsize=(10,4))
# plt.set_title('Training & Validation Loss')
no_of_features = list(range(0, len(avg_random) ))
plt.plot( no_of_features, avg_random, color='navy', marker='*', linestyle='-', label='Random')
plt.plot( no_of_features, avg_shap, color='green', marker='x', linestyle='-', label='Shap')
plt.plot( no_of_features, avg_lime, color='red', marker='o', linestyle='-', label='Lime')
plt.title("Average time taken for each explainer to compute top 30 most important features of a row based on the nsamples allowed to compute")
plt.legend(loc='best')

In [ ]:
avg_shap